In [2]:
import numpy as np
import tensorflow as tf
import pandas as pd
from ipywidgets import widgets
from sklearn import preprocessing
from tensorflow import keras 
from tensorflow.keras import layers, regularizers,Model, utils
%matplotlib inline 
import matplotlib.pyplot as plt
import sklearn
from tensorflow.keras.callbacks import TensorBoard
from datetime import time
import plotly.express as px
from sklearn.model_selection import train_test_split
import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path
import plotly.express as px
from scipy.ndimage.filters import gaussian_filter
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from livelossplot import PlotLossesKeras

from os import listdir
import os
#thanks to https://stackoverflow.com/users/218681/bernhard-kausler
def find_csv_filenames( path_to_dir, suffix=".csv" ):
    filenames = listdir(path_to_dir)
    return [ filename for filename in filenames if filename.endswith( suffix ) ]

In [23]:
drivePaths = find_csv_filenames("outputs")[0:1]
subsamplingPeriod = 32
sequenceLength = 16
smoothingSigma = 2

In [24]:
drivesWithLocation = []
drivesWithoutLocation = []
for drivePath in drivePaths:
    drive = pd.read_csv("outputs/"+drivePath)
    print(drive)
    drive = drive.iloc[::subsamplingPeriod].copy(deep=True)
    driveWithoutLocation = drive.drop(columns=["Time", "Longitude", "Latitude"])
    drivesWithLocation.append(drive)
    drivesWithoutLocation.append(driveWithoutLocation)

               Time   Longitude   Latitude      Speed  LatAcceleration  \
0      1.616265e+09 -111.051270  32.376080   0.000000         1.000000   
1      1.616265e+09 -111.051262  32.376083   0.000000         0.082703   
2      1.616265e+09 -111.051254  32.376083   0.000000         1.000000   
3      1.616265e+09 -111.051254  32.376083   0.000000         1.000000   
4      1.616265e+09 -111.051247  32.376083   0.000000         1.000000   
...             ...         ...        ...        ...              ...   
17793  1.616267e+09 -110.845238  32.308804  35.520804         5.215355   
17794  1.616267e+09 -110.845222  32.308804  34.932791         5.683811   
17795  1.616267e+09 -110.845215  32.308804  34.708273         4.006824   
17796  1.616267e+09 -110.845200  32.308804  34.164935         7.000000   
17797  1.616267e+09 -110.845192  32.308804  33.721619         4.038464   

       LongAcceleration  ZAcceleration  YawRate  SteerTorque  SteerRate  \
0              0.000000      46.0000

In [25]:
normalizedDrives = []
for drive in drivesWithoutLocation:
    standard_scaler = preprocessing.StandardScaler()
    data_normalized = standard_scaler.fit_transform(drive)
    data_normalized = pd.DataFrame(data_normalized)
    normalizedDrives.append(data_normalized.to_numpy())

In [26]:
normalizedDrives

[array([[-1.47386292e+00, -1.79647769e-01, -6.51513530e-03, ...,
         -1.47493146e+00, -1.47411348e+00, -1.47460123e+00],
        [-1.47386292e+00, -1.79647769e-01, -6.51513530e-03, ...,
         -1.47493146e+00, -1.47411348e+00, -1.47460123e+00],
        [-1.47386292e+00, -1.79647769e-01, -6.51513530e-03, ...,
         -1.47493146e+00, -1.47411348e+00, -1.47460123e+00],
        ...,
        [ 4.71883404e-01,  1.45991609e-01, -2.58078763e+00, ...,
          4.66583077e-01,  4.64451014e-01,  4.61193621e-01],
        [ 5.30947032e-02,  3.87382802e-01, -1.38808268e+00, ...,
          4.98924770e-02,  5.49326454e-02,  4.97179716e-02],
        [-2.85436619e-01, -2.59614325e-04, -2.22868045e+00, ...,
         -2.89566279e-01, -2.85116812e-01, -2.89190940e-01]])]

In [27]:
roughData = np.vstack(normalizedDrives[:])
print(roughData.shape)

(557, 13)


for k in range(len(drivePaths)):
    print(drivePath[k])
    roughData = normalizedDrives[k]
    smoothData = -1*np.ones(shape=roughData.shape)
    features = range(roughData.shape[1])
    for curFeature in features:
        print("smoothing %s"%curFeature)

        smoothData[:,curFeature] = gaussian_filter(roughData[:,curFeature], sigma=smoothingSigma)#.rolling(window=sequenceLength, win_type='gaussian', center=True).mean(std=0.5)
    #     ewmaSmoothData[curFeature] = pd.DataFrame.ewm(data_normalized[curFeature], span=sequenceLength)

        #plot the original feature and the smoothed feature
        scatterData = pd.DataFrame({
        "index":range(roughData.shape[0]),
        "smoothData": smoothData[:,curFeature],
        #"ewmaSmoothData": ewmaSmoothData[curFeature],
        "originalData": roughData[:,curFeature],
        })
        scatterData.describe()

        fig1 = go.Figure()
    #     fig1.add_trace(go.Scatter(x=scatterData.index, y=scatterData.ewmaSmoothData, name="ewma smoothed data"))
        fig1.add_trace(go.Scatter(x=scatterData.index, y=scatterData.smoothData, name="gaussian smoothed data"))
        fig1.add_trace(go.Scatter(x=scatterData.index, y=scatterData.originalData, name="original data"))

        fig1.show()
    
    
#